In [1]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import warnings

In [2]:
'''
Sliding window attempt: Not Successful

'''
warnings.filterwarnings('ignore') 

def csv_extract(filename = "/NAVER_DATA.csv"):
    '''
    Input: Number of samples, Number of Stocks, filename
    Output: numpy array of (number of samples, number of stocks)
    '''
  
    df = pd.read_csv(filename)
    df['Roc'] = (df['Roc'] - 100)/2
    df['Momentum'] = df['Momentum']/df['Price']
    df['RSI'] = 2*df['RSI'] - 1 
    df['CCI'] = df['CCI']/100
    df['ADX'] = 2*df['ADX'] - 1
    df['TRIX'] = 10*df['TRIX']
    df['MACD'] = np.log(np.abs(df['MACD'])) * np.sign(df['MACD'])
    df['OBV'] = np.log(np.abs(df['OBV'])) * np.sign(df['OBV'])
    df['_TSF_'] = np.log(np.abs(df['_TSF_'])) * np.sign(df['_TSF_'])
    df['ATR'] = np.log(np.abs(df['ATR'])) * np.sign(df['ATR'])
    df['MFI'] = 2*df['MFI']/100 - 1
    day_num = 1
    X = np.empty((600 - day_num, 0))
    y = np.empty(0)
    for n in range(df['Price'].shape[0] - day_num):
        temp = 1 if df['Price'][n + day_num] >= df['Price'][n] else 0
        if df['Price'][n + day_num] > df['Price'][n]:
            temp = 1
        elif  df['Price'][n + day_num] < df['Price'][n]:
            temp = 0
        else:
            temp = 2
        y = np.append(y, temp)
    remove = ['Price'] 
    #remove = ['Price', '_TSF_', 'MFI', 'MACD', 'OBV', 'Momentum', 'Roc', 'RSI', 'TRIX']
    #remove = ['Momentum', 'RSI', 'MACD', 'OBV', 'ADX', 'MFI'] 
    remove = []
    for row in df:
        if row in remove:
            continue
        X = np.column_stack((X, df[row][0:600 - day_num]))
    print(df)
    #X = preprocessing.scale(X)
    return X, y

filename = "./NAVER_DATA.csv"
X, y = csv_extract(filename = filename)
X = X[31:]
y = y[31:]
verbose = False
d = {}

for window in np.arange(5, 100, 5):
    print('Window Size: %d' %(window))
    tot_right = 0; tot = 0
    y_pred_ = []
    y_test_ = []
    for n in range(y.shape[0] - window):
        if n%10 == 0:
            print('%f completed' %(100*n/(y.shape[0] - window)))
        X_train, y_train = X[n:n+window-1], y[n:n+window-1]
        X_test, y_test = np.resize(X[n+window], (1, X.shape[1])), y[n+window]
        #tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-2, 1e-1, 1e+0, 1e+1],
        #              'C': [1e+0, 1e+1, 1e+2, 1e+3, 1e+4]}]
        tuned_parameters = [{'kernel': ['linear'], 'C': [1e+0, 1e+1]}]
        score = 'recall'
        '''clf = GridSearchCV(
            SVC(), tuned_parameters, scoring='%s_macro' % score
        )'''
        clf = SVC(kernel = 'linear', C = 10) 
        try:
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
        except Exception:
            #print(y_train)
            y_pred = y_train[0]
        tot = tot + 1
        tot_right = tot_right + 1 if y_pred == y_test else tot_right
        try:
            y_pred_.append(y_pred[0])
        except Exception:
            y_pred_.append(y_pred)
        y_test_.append(y_test)
        if verbose:
            print("Best parameters set found on development set:")
            print()
            print(clf.best_params_)
            print()
            print("Grid scores on development set:")
            print()
            means = clf.cv_results_['mean_test_score']
            stds = clf.cv_results_['std_test_score']
            for mean, std, params in zip(means, stds, clf.cv_results_['params']):
                print("%0.3f (+/-%0.03f) for %r"
                % (mean, std * 2, params))
    d[window] = tot_right/tot
    print(window, tot_right/tot)
    print(classification_report(y_test_, y_pred_))

v=list(d.values())
k=list(d.keys())
print(max(v), k[v.index(max(v))])

      Price  William's %R     Roc  Momentum     RSI     CCI     ADX  TRIX  \
0    274500        0.0000 -50.000  0.000000 -1.0000  0.0000 -1.0000   0.0   
1    278500        0.0000 -50.000  0.000000 -1.0000  0.0000 -1.0000   0.0   
2    276500       -0.2500 -50.000  0.000000 -1.0000  0.0000 -1.0000   0.1   
3    275500       -0.3750 -50.000  0.000000 -1.0000  0.0000 -1.0000   0.1   
4    275500       -0.3750 -50.000  0.000000 -1.0000  0.0000 -1.0000   0.2   
..      ...           ...     ...       ...     ...     ...     ...   ...   
595  317500       -0.2105   0.720  0.012598  0.2666  0.5372 -0.0354   0.9   
596  318500       -0.1053   0.795  0.014129  0.2904  0.6458 -0.0198   0.9   
597  319000       -0.0526   1.040  0.018809  0.4000  0.8410  0.0092   1.0   
598  319500       -0.0500   0.795  0.018779  0.4000  1.1561  0.0586   1.0   
599  320000        0.0000   0.475  0.023438  0.5172  1.5659  0.1014   1.0   

         MACD        OBV      _TSF_       ATR     MFI  
0         NaN      

40.515654 completed
42.357274 completed
44.198895 completed
46.040516 completed
47.882136 completed
49.723757 completed
51.565378 completed
53.406998 completed
55.248619 completed
57.090239 completed
58.931860 completed
60.773481 completed
62.615101 completed
64.456722 completed
66.298343 completed
68.139963 completed
69.981584 completed
71.823204 completed
73.664825 completed
75.506446 completed
77.348066 completed
79.189687 completed
81.031308 completed
82.872928 completed
84.714549 completed
86.556169 completed
88.397790 completed
90.239411 completed
92.081031 completed
93.922652 completed
95.764273 completed
97.605893 completed
99.447514 completed
25 0.39042357274401474
              precision    recall  f1-score   support

         0.0       0.42      0.57      0.48       225
         1.0       0.41      0.31      0.35       228
         2.0       0.21      0.14      0.17        90

    accuracy                           0.39       543
   macro avg       0.34      0.34      0.34  

1.949318 completed
3.898635 completed
5.847953 completed
7.797271 completed
9.746589 completed
11.695906 completed
13.645224 completed
15.594542 completed
17.543860 completed
19.493177 completed
21.442495 completed
23.391813 completed
25.341131 completed
27.290448 completed
29.239766 completed
31.189084 completed
33.138402 completed
35.087719 completed
37.037037 completed
38.986355 completed
40.935673 completed
42.884990 completed
44.834308 completed
46.783626 completed
48.732943 completed
50.682261 completed
52.631579 completed
54.580897 completed
56.530214 completed
58.479532 completed
60.428850 completed
62.378168 completed
64.327485 completed
66.276803 completed
68.226121 completed
70.175439 completed
72.124756 completed
74.074074 completed
76.023392 completed
77.972710 completed
79.922027 completed
81.871345 completed
83.820663 completed
85.769981 completed
87.719298 completed
89.668616 completed
91.617934 completed
93.567251 completed
95.516569 completed
97.465887 completed
99.41

2.070393 completed
4.140787 completed
6.211180 completed
8.281573 completed
10.351967 completed
12.422360 completed
14.492754 completed
16.563147 completed
18.633540 completed
20.703934 completed
22.774327 completed
24.844720 completed
26.915114 completed
28.985507 completed
31.055901 completed
33.126294 completed
35.196687 completed
37.267081 completed
39.337474 completed
41.407867 completed
43.478261 completed
45.548654 completed
47.619048 completed
49.689441 completed
51.759834 completed
53.830228 completed
55.900621 completed
57.971014 completed
60.041408 completed
62.111801 completed
64.182195 completed
66.252588 completed
68.322981 completed
70.393375 completed
72.463768 completed
74.534161 completed
76.604555 completed
78.674948 completed
80.745342 completed
82.815735 completed
84.886128 completed
86.956522 completed
89.026915 completed
91.097308 completed
93.167702 completed
95.238095 completed
97.308489 completed
99.378882 completed
85 0.40372670807453415
              precisi